# Elasticsearch With Haystack

We will be communicating with our Elasticsearch document store via Haystack. First, we need to install Haystack using pip:

On Windows:

```
pip install farm-haystack -f https://download.pytorch.org/whl/torch_stable.html
```

Anything else:

```
pip install farm-haystack
```

We will start by indexing the SQuAD dev data. So let's load that into our notebook first.

